In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://rever.vn/s/ho-chi-minh/mua/nha-pho?page="
session = requests.Session()

house_data_list = []

for page in range(1, 500):
    response = session.get(url + str(page))
    soup = BeautifulSoup(response.content, "html.parser")
    productslists = soup.select(".col.property-itemz a.listing-price-link[href]")

    for link in productslists:
        product_url = link['href']
        r = session.get(product_url)
        soup = BeautifulSoup(r.content, 'html.parser')

        # Lấy thông tin địa chỉ từ thẻ address
        address_div = soup.select_one("div.address")
        duong = ""
        phuong = ""
        quan = ""
        
        if address_div:
            # Duyệt qua các thẻ <a> trong div.address để lấy thông tin đường, phường, quận
            for a_tag in address_div.find_all('a'):
                title = a_tag.get('title')
                text = a_tag.get_text()

                if title == "Đường":
                    duong = text
                elif title == "Phường":
                    phuong = text
                elif title == "Quận":
                    quan = text
        
        code_update = soup.select_one("div.listing-id-status").get_text()

        # Lấy thông tin từ ul.detail-more
        basic_info = soup.select_one("ul.detail-more")
        loai_hinh = ""
        so_phong_ngu = ""
        so_phong_tam = ""
        so_tang = ""
        dien_tich_dat = ""
        dien_tich_xay_dung = ""
        dien_tich_san = ""
        tinh_hinh_noi_that = ""
        loai_chu_quyen = ""
        tinh_trang_su_dung = ""
        du_an = ""
        gia_ban = ""

        if basic_info:
            loai_hinh = basic_info.find('p', text="Loại hình").find_next_sibling('p').get_text(strip=True) if basic_info.find('p', text="Loại hình") else ""
            so_phong_ngu = basic_info.find('p', text="Phòng ngủ").find_next_sibling('p').get_text(strip=True) if basic_info.find('p', text="Phòng ngủ") else "1"
            so_phong_tam = basic_info.find('p', text="Phòng tắm").find_next_sibling('p').get_text(strip=True) if basic_info.find('p', text="Phòng tắm") else "1"
            so_tang = basic_info.find('p', text="Số tầng").find_next_sibling('p').get_text(strip=True) if basic_info.find('p', text="Số tầng") else "1"
            tinh_hinh_noi_that = basic_info.find('p', text="Tình hình nội thất").find_next_sibling('p').get_text(strip=True) if basic_info.find('p', text="Tình hình nội thất") else "Không có nội thất"
            loai_chu_quyen = basic_info.find('p', text="Loại chủ quyền").find_next_sibling('p').get_text(strip=True) if basic_info.find('p', text="Loại chủ quyền") else ""
            tinh_trang_su_dung = basic_info.find('p', text="Tình trạng sử dụng").find_next_sibling('p').get_text(strip=True) if basic_info.find('p', text="Tình trạng sử dụng") else "Nhà trống"
            du_an = basic_info.find('p', text="Dự án").find_next_sibling('p').get_text(strip=True) if basic_info.find('p', text="Dự án") else ""
            
        # Lấy thông tin từ thẻ mới bổ sung (ul.detailroom)
        detailroom = soup.select_one("ul.detailroom")
        price = ""
        price_per_m2 = ""
        bedrooms = ""
        bathrooms = ""
        area = ""
        house_orientation = ""

        if detailroom:
            # Lấy giá và giá/m2
            price = detailroom.select_one("div.listing-detail-price-cost strong").get_text() if detailroom.select_one("div.listing-detail-price-cost strong") else ""

            # Lấy diện tích
            area = detailroom.find('li', title="Diện tích").get_text(strip=True) if detailroom.find('li', title="Diện tích") else ""

            # Lấy hướng nhà
            house_orientation = detailroom.find('li', title="Hướng nhà").get_text(strip=True) if detailroom.find('li', title="Hướng nhà") else ""

        summary = soup.select_one("div.content.collapsed.pdb-0 p.summary").get_text(strip=True)

        # Lấy thông tin tiện ích từ ul.detail-more.utilities-list
        utilities_list = soup.select_one("ul.detail-more.utilities-list")
        utilities = ""
        if utilities_list:
            utilities = ', '.join([li.get_text(strip=True) for li in utilities_list.find_all('li')])

        # Lưu thông tin vào dictionary
        house_data = {
            "Đường": duong,
            "Phường": phuong,
            "Quận": quan,
            "Loại hình": loai_hinh,
            "Phòng ngủ": so_phong_ngu,
            "Phòng tắm": so_phong_tam,
            "Số tầng": so_tang,
            "Diện tích": area,
            "Tình hình nội thất": tinh_hinh_noi_that,
            "Loại chủ quyền": loai_chu_quyen,
            "Tình trạng sử dụng": tinh_trang_su_dung,
            "Dự án": du_an,
            "Giá": price,
            "Hướng nhà": house_orientation,

        }
        house_data_list.append(house_data)

df = pd.DataFrame(house_data_list)

C:\Users\CUONG DAT PHAT\AppData\Local\Temp\ipykernel_12024\2886476464.py:57: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  loai_hinh = basic_info.find('p', text="Loại hình").find_next_sibling('p').get_text(strip=True) if basic_info.find('p', text="Loại hình") else ""
C:\Users\CUONG DAT PHAT\AppData\Local\Temp\ipykernel_12024\2886476464.py:58: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  so_phong_ngu = basic_info.find('p', text="Phòng ngủ").find_next_sibling('p').get_text(strip=True) if basic_info.find('p', text="Phòng ngủ") else "1"
C:\Users\CUONG DAT PHAT\AppData\Local\Temp\ipykernel_12024\2886476464.py:59: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  so_phong_tam = basic_info.find('p', text="Phòng tắm").find_next_sibling('p').get_text(strip=True) if basic_info.find('p', text="Phòng tắm") else "1"
C:\Users\CUONG DAT P

In [6]:
df

,Đường,Phường,Quận,Loại hình,Phòng ngủ,Phòng tắm,Số tầng,Diện tích,Tình hình nội thất,Loại chủ quyền,Tình trạng sử dụng,Dự án,Giá,Hướng nhà
0,,An Phú,Quận 2,Nhà phố,4,4,5,95m²,Không có nội thất,HĐ mua bán,Nhà trống,The Global City,36 tỷ,Tây Bắc
1,,An Phú,Quận 2,Nhà phố,4,4,5,95m²,Không có nội thất,HĐ mua bán,Nhà trống,The Global City,33 tỷ,Tây Bắc
2,,An Phú,Quận 2,Nhà phố,4,4,5,95m²,Không có nội thất,HĐ mua bán,Nhà trống,The Global City,36 tỷ,Đông Nam
3,,An Phú,Quận 2,Nhà phố,4,4,5,95m²,Không có nội thất,HĐ mua bán,Nhà trống,The Global City,32.5 tỷ,Tây Bắc
4,Nguyễn Xiển,Long Thạnh Mỹ,Quận 9,Nhà phố,4,4,5,144m²,Nội thất cơ bản,HĐ mua bán,Nhà trống,Vinhomes Grand Park,19 tỷ,
5,Nguyễn Xiển,Long Thạnh Mỹ,Quận 9,Nhà phố,5,4,5,84m²,Không có nội thất,HĐ mua bán,Nhà trống,Vinhomes Grand Park,12.8 tỷ,Tây Bắc
6,Nguyễn Xiển,Long Thạnh Mỹ,Quận 9,Nhà phố,4,4,3,250m²,Không có nội thất,HĐ mua bán,Nhà trống,Vinhomes Grand Park,28.6 tỷ,Tây Bắc
7,Nguyễn Xiển,Long Thạnh Mỹ,Quận 9,Nhà phố,3,4,4,200m²,Không có nội thất,HĐ mua bán,Nhà trống,Vinhomes Grand Park,24.5 tỷ,Đông Nam
8,Trần Quang Khải,Tân Định,Quận 1,Nhà phố,2,4,3,39.3m²,Đầy đủ nội thất,Sổ hồng,Nhà trống,,5.8 tỷ,Đông Nam
9,Rạch Bùng Binh,Phường 10,Quận 3,Nhà phố,2,3,3,18m²,Đầy đủ nội thất,Sổ hồng,Đang cho thuê,,3 tỷ,Tây Bắc


In [4]:
df.to_excel('D:/du lieu o cu/Statistics and Machine Learning Course/Đồ án/data_nhapho.xlsx')